In [1]:
import image
import math
import sys


originalImage = "flower.jpeg"


def pixelMapper(old, rgbFunction):

    width = old.getWidth()
    height = old.getHeight()
    new = image.EmptyImage(width, height)

    for row in range(height):
        for col in range(width):
            pixel = old.getPixel(col, row)
            new.setPixel(col, row, rgbFunction(pixel))

    return new


def imageTransform(imageFile, rgbFunction):

    old = imageFile
    width = old.getWidth()
    height = old.getHeight()

    new = pixelMapper(old, rgbFunction)

    return new


# vertical axis flip

def verticalFlip(originalImage):
    height = originalImage.getHeight()
    width = originalImage.getWidth()
    last = width - 1

    flippedImage = image.EmptyImage(width, height)
    for x in range(width):
        for y in range(height):
            pixel = originalImage.getPixel(last - x, y)
            flippedImage.setPixel(x, y, pixel)

    return flippedImage


# sepia (requires imageTransform)

def sepia_pixel(pixel):
    new_r = min(round(pixel.getRed() * 0.393 + pixel.getGreen()
                      * 0.796 + pixel.getBlue() * 0.189), 255)
    new_g = min(round(pixel.getRed() * 0.349 + pixel.getGreen()
                      * 0.686 + pixel.getBlue() * 0.168), 255)
    new_b = min(round(pixel.getRed() * 0.272 + pixel.getGreen()
                      * 0.534 + pixel.getBlue() * 0.131), 255)

    new_pixel = image.Pixel(new_r, new_g, new_b)
    return new_pixel


# mirror

def mirror(originalImage):
    height = originalImage.getHeight()
    width = originalImage.getWidth()

    mirroredImage = image.EmptyImage(width, height)

    for x in range(width//2):
        for y in range(height):
            pixel = originalImage.getPixel(width - 1 - x, y)
            mirroredImage.setPixel(x, y, pixel)
            mirroredImage.setPixel(width - 1 - x, y, pixel)

    return mirroredImage


# grayscale (req imageTransform)

def grayscale_pixel(pixel):
    totIntensity = pixel.getRed() + pixel.getGreen() + pixel.getBlue()
    avgIntensity = totIntensity // 3
    pixel = image.Pixel(avgIntensity, avgIntensity, avgIntensity)
    return pixel


# negative (requires imageTransform)

def negative_pixel(oldPixel):
    newRed = 255 - oldPixel.getRed()
    newGreen = 255 - oldPixel.getGreen()
    newBlue = 255 - oldPixel.getBlue()
    newPixel = image.Pixel(newRed, newGreen, newBlue)
    return newPixel


# edge detection

def edge_convolve(originalImage, pixelRow, pixelCol, kernel):
    kernelColBase = pixelCol - 1
    kernelRowBase = pixelRow - 1
    total = 0
    for row in range(kernelRowBase, kernelRowBase+3):
        for col in range(kernelColBase, kernelColBase+3):
            kColIndex = col - kernelColBase
            kRowIndex = row - kernelRowBase
            pixel = originalImage.getPixel(col, row)
            intensity = pixel.getRed()
            total = total + intensity * kernel[kRowIndex][kColIndex]
    return total


def edgeDetect(originalImage):

    grayscale = imageTransform(originalImage, grayscale_pixel)

    edgeImage = image.EmptyImage(
        originalImage.getWidth(), originalImage.getHeight())
    black = image.Pixel(0, 0, 0)
    white = image.Pixel(254, 254, 254)
    xMask = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
    yMask = [[1, 0, -1], [2, 0, -2], [1, 0, -1]]

    for row in range(1, originalImage.getHeight()-1):
        for col in range(1, originalImage.getWidth()-1):
            gX = edge_convolve(grayscale, row, col, xMask)
            gY = edge_convolve(grayscale, row, col, yMask)
            g = math.sqrt(gX**2 + gY**2)

            if g > 175:
                edgeImage.setPixel(col, row, white)

            else:
                edgeImage.setPixel(col, row, black)

    return edgeImage


# choice (req imageTransform)

def swap_colors(old):
    new = image.Pixel(old.getGreen(), old.getBlue(), old.getRed())
    return new


# blur

def blur_convolve(originalImage, pixelRow, pixelCol, kernel):

    kernelColBase = pixelCol - 1
    kernelRowBase = pixelRow - 1
    new = 0

    r_intensity = 0
    g_intensity = 0
    b_intensity = 0
    for row in range(kernelRowBase, kernelRowBase+3):
        for col in range(kernelColBase, kernelColBase+3):
            kColIndex = col - kernelColBase
            kRowIndex = row - kernelRowBase
            multiplier = kernel[kRowIndex][kColIndex]
            pixel = originalImage.getPixel(col, row)
            r_intensity += pixel.getRed() * multiplier
            g_intensity += pixel.getGreen() * multiplier
            b_intensity += pixel.getBlue() * multiplier
    new = image.Pixel(int(r_intensity), int(g_intensity), int(b_intensity))

    return new


def blur(originalImage):

    blurImage = image.EmptyImage(
        originalImage.getWidth(), originalImage.getHeight())
    mask = [[1/13, 2/13, 1/13], [2/13, 1/13, 2/13], [1/13, 2/13, 1/13]]

    for row in range(1, originalImage.getHeight()-1):
        for col in range(1, originalImage.getWidth()-1):
            color = blur_convolve(originalImage, row, col, mask)
            blurImage.setPixel(col, row, color)

    return blurImage


########################

def black_pixel(width, height):

    black = image.EmptyImage(width, height)
    b = image.Pixel(0, 0, 0)
    for row in range(width):
        for col in range(height):
            black.setPixel(row, col, b)

    return black


# making the collage

def collage(imageFile):

    buffer = 4

    originalImage = image.FileImage(imageFile)
    height = originalImage.getHeight()
    width = originalImage.getWidth()

    window = image.ImageWin(width * 3 + (buffer * 4),
                            height * 3 + (buffer * 4), "Collage Project")

    black = black_pixel(width * 3 + (buffer * 4), height * 3 + (buffer * 4))
    black.draw(window)

    originalImage.setPosition(width + (buffer * 2), height + (buffer * 2))
    originalImage.draw(window)

    horizontal = verticalFlip(originalImage)
    horizontal.setPosition(buffer, buffer)
    horizontal.draw(window)

    sepia = imageTransform(originalImage, sepia_pixel)
    sepia.setPosition(width + (buffer * 2), buffer)
    sepia.draw(window)

    mirrored = mirror(horizontal)
    mirrored.setPosition(width * 2 + (buffer * 3), buffer)
    mirrored.draw(window)

    grayscale = imageTransform(originalImage, grayscale_pixel)
    grayscale.setPosition(buffer, height + (buffer * 2))
    grayscale.draw(window)

    negative = imageTransform(originalImage, negative_pixel)
    negative.setPosition(width * 2 + (buffer * 3), height + (buffer * 2))
    negative.draw(window)

    edge = edgeDetect(originalImage)
    edge.setPosition(buffer, height * 2 + (buffer * 3))
    edge.draw(window)

    choice = imageTransform(originalImage, swap_colors)
    choice.setPosition(width + (buffer * 2), height * 2 + (buffer * 3))
    choice.draw(window)

    blurred = blur(originalImage)
    blurred.setPosition(width * 2 + (buffer * 3), height * 2 + (buffer * 3))
    blurred.draw(window)

    window.exit_on_click()


collage(originalImage)


In [2]:
import image

originalImage = image.FileImage("flower.jpeg")

height = originalImage.getHeight()
width = originalImage.getWidth()

choice2 = imageTransform(originalImage, grayscale_pixel)
choice2 = imageTransform(choice2, negative_pixel)

window2 = image.ImageWin(height, width)

choice2.draw(window2)